In [11]:
from py_clob_client.client import ClobClient
from pathlib import Path


host: str = "https://clob.polymarket.com"
key: str = Path("/Users/kate/.secrets/polymarket_key").read_text().strip() 
chain_id: int = 137 #No need to adjust this

### Initialization of a client that trades directly from an EOA. 
# client = ClobClient(host, key=key, chain_id=chain_id)

client = ClobClient(host, key=key, chain_id=chain_id, signature_type=1, funder="0x17eaa362893553bc35ff7955f3df341255f07838")


# print("creating...")
# print(client.create_api_key())

print("deriving...")
api_creds = client.derive_api_key()

# client.create_order

deriving...


In [12]:
import json


api_creds_dict = {
    'api_key': api_creds.api_key,
    'api_secret': api_creds.api_secret,
    'api_passphrase': api_creds.api_passphrase
}

print(json.dumps(api_creds_dict, indent=2))

api_creds_path = Path("/Users/kate/.secrets/api_creds.json")
with open(api_creds_path, 'w') as f:
    json.dump(api_creds_dict, f, indent=2)

{
  "api_key": "01bad0c0-a9ed-535b-6400-c10628f371d1",
  "api_secret": "B4rOK0swEbEus7FbMHjKACHVpi6u-pTy68FrOaMTlrE=",
  "api_passphrase": "fc85c2c123ef94e79235cb75f9284d8c3c342206223f1dc2f29aea4d1a2bb934"
}


In [ ]:
from py_clob_client.clob_types import OrderArgs, OrderType, PartialCreateOrderOptions, TickSize
from py_clob_client.order_builder.constants import BUY


order_args = OrderArgs(
    price=0.039,
    size=75,
    side=BUY,
    token_id="94590870645398328591006040214534041508719638068332637015636239267133245387890",
)
signed_order = client.create_order(order_args, 
                                   options=PartialCreateOrderOptions(neg_risk=True,
                                                                     tick_size="0.001"))

print("Signed Order: {signed_order}")

Signed Order: {signed_order}


In [40]:
from py_clob_client.clob_types import OpenOrderParams

client.set_api_creds(api_creds)

resp = client.get_orders(
    OpenOrderParams()
)

resp

[]

In [41]:
from py_clob_client.client import order_to_json

order_to_json(signed_order, client.creds.api_key, OrderType.FAK)

{'order': {'salt': 65484531,
  'maker': '0x17EaA362893553bC35Ff7955f3Df341255F07838',
  'signer': '0x876c93fF619596c24C00Ce512b41C221a40C6e81',
  'taker': '0x0000000000000000000000000000000000000000',
  'tokenId': '94590870645398328591006040214534041508719638068332637015636239267133245387890',
  'makerAmount': '2925000',
  'takerAmount': '75000000',
  'expiration': '0',
  'nonce': '0',
  'feeRateBps': '0',
  'side': 'BUY',
  'signatureType': 1,
  'signature': '0x8d361093a220b75d47d00af01e30f3346880e07ea858dd039176d903b474b3be0a2df3627edf6981e1011c37173a96b5d66dfa150bc506d512e7b3065df5cd2e1c'},
 'owner': '01bad0c0-a9ed-535b-6400-c10628f371d1',
 'orderType': 'FAK'}

In [35]:
client.post_order(signed_order, orderType=OrderType.FAK)

PolyApiException: PolyApiException[status_code=400, error_message={'error': 'not enough balance / allowance'}]